In [35]:
####################################Imports and Initial Feed Rip###################################################
import feedparser as fp, pandas as pd, datetime, time, string, requests as r
from bs4 import BeautifulSoup 
from functools import reduce

##TODO - Input these variables

rss_feed = 'http://feeds.huffingtonpost.com/c/35496/f/677097/index.rss'
company_name = 'Apple'
ticker_symbol = 'AAPL'


#Get the News Feed and Slice it to only include the news items
feed = fp.parse(rss_feed)
feed_items = feed['items']

###################################################################################################################

In [36]:
######################################Write the Date&Time that the Rip Occured#####################################
download_time = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')

file = open('dateDownloaded.txt','w')
file.write(download_time)
file.close()


###################################################################################################################

In [37]:
#################################Create the Cleaned News Items DataFrame###########################################

#Import the raw data into a DataFrame
raw_news_dataframe = pd.DataFrame(feed_items)

#Clean the DataFrame
def dataframe_cleaner(dataframe): 
    being_cleaned_df = pd.DataFrame(raw_news_dataframe[['author','id','title','summary']])
    
    for i in range(0,len(being_cleaned_df['summary'])):
        soup = BeautifulSoup(being_cleaned_df['summary'][i],'lxml')
        being_cleaned_df['summary'][i] = soup.get_text()
    
    return being_cleaned_df
    
#Create the cleaned Dataframe    
news_df = dataframe_cleaner(raw_news_dataframe)
news_df.head(1)

###################################################################################################################

,author,id,title,summary
0,Sari Gabbay,http://www.huffingtonpost.com/sari-gabbay/the-...,The Roars of Cannons Wont Stop the Explosions ...,If you are reading this and planning on comm...


In [38]:
#Load in the Sentiment DataFrame
sentiment_df = pd.read_csv('sentiment_DataFrame.csv')
sentiment_df = sentiment_df.ix[:,'type':]
sentiment_df.head(1)

,type,word,position,stemmed,polarity
0,0.5,abandoned,adj,n,-1


In [39]:
##########################Perform Sentiment Analysis on the Title and Body of the Articles#########################
def title_analysis(title):
    tokenized_title = dict.fromkeys(title.split(' '))                          
                                
    for key in tokenized_title:
        tokenized_title[key] = [sentiment_df['type'][i] * sentiment_df['polarity'][i]
                               for i in range(0,len(sentiment_df.index)) 
                               if key.lower() == sentiment_df['word'][i]]
    
    total_sentiment = sum([item for sublist in list(tokenized_title.values()) for item in sublist])
    normalized_sentiment = total_sentiment / len(tokenized_title)
    
    return normalized_sentiment

def summary_analysis(summary):
    tokenized_summary = dict.fromkeys(summary.split(' '))
    
    for key in tokenized_summary:
        tokenized_summary[key] = [sentiment_df['type'][i] * sentiment_df['polarity'][i]
                               for i in range(0,len(sentiment_df.index)) 
                               if key.lower() == sentiment_df['word'][i]]
    
    total_sentiment = sum([item for sublist in list(tokenized_summary.values()) for item in sublist])
    normalized_sentiment = total_sentiment / len(tokenized_summary)
    return normalized_sentiment

def sentiment_analysis(dataframe):
    analyzing_df = dataframe
    analyzing_df['normalized_title_sentiment'] = list(map(lambda x: title_analysis(x),analyzing_df['title']))
    analyzing_df['normalized_summary_sentiment'] = list(map(lambda x: summary_analysis(x),analyzing_df['summary']))
    return analyzing_df

start_time = time.time()
analyzed_articles_df = sentiment_analysis(news_df)
elapsed_time = time.time() - start_time
print('Elapsed Time: ' + str(elapsed_time))
analyzed_articles_df.head(1)

###################################################################################################################

Elapsed Time: 460.58625626564026


,author,id,title,summary,normalized_title_sentiment,normalized_summary_sentiment
0,Sari Gabbay,http://www.huffingtonpost.com/sari-gabbay/the-...,The Roars of Cannons Wont Stop the Explosions ...,If you are reading this and planning on comm...,0,0.097686


In [33]:
################################################Pull Stock Data#####################################################
raw_stock_quote = r.get('http://dev.markitondemand.com/Api/v2/Quote/json?symbol=' + ticker_symbol).json()
stock_quote = pd.DataFrame(raw_stock_quote,index=[0])
stock_quote.head()
###################################################################################################################

,Change,ChangePercent,ChangePercentYTD,ChangeYTD,High,LastPrice,Low,MSDate,MarketCap,Name,Open,Status,Symbol,Timestamp,Volume
0,-1.32,-1.187905,4.313129,105.26,110.73,109.8,109.43,42465.665972,608795213400,Apple Inc,109.59,SUCCESS,AAPL,Tue Apr 5 15:59:00 UTC-04:00 2016,2015632


In [49]:
filter_list = list(map(lambda x: company_name in x,analyzed_articles_df['title']))
filtered_dataframe = analyzed_articles_df[filter_list]
filtered_dataframe.head()

,author,id,title,summary,normalized_title_sentiment,normalized_summary_sentiment
